Data wrangling / dat cleaning of the Brussels metropolitan area. Detailed report: brussels_report.md

In [3]:
# original source file downloaded from https://s3.amazonaws.com/metro-extracts.mapzen.com/brussels_belgium.osm.bz2

bxl = 'brussels_belgium.osm'

# Exploration and preparing for export

In [1]:
# imprort all fucntions from file wrangled_brussels.py

from wrangled_brussels import *

## Encoding

In [35]:
# fix unicode for street and cities (mostly) for French names

fix_osm(bxl, fix_unicode, 'name:fr')
fix_osm(bxl, fix_unicode, 'name')
fix_osm(bxl, fix_unicode, 'addr:street')
fix_osm(bxl, fix_unicode, 'addr:city')
fix_osm(bxl, fix_unicode, 'addr:city:fr')
fix_osm(bxl, fix_unicode, 'addr:street:fr')
fix_osm(bxl, fix_unicode, 'title')

In [6]:
# Number of tags with different keys and values for cities and streets - before editing. More details in the report 
# (osm_brussels_Report.mk file).

process_names(bxl)

defaultdict(int,
            {'addr:city:bi': 43997,
             'addr:city:fr': 7285,
             'addr:city:mono': 41350,
             'addr:city:nl': 7285,
             'addr:street:bi': 221764,
             'addr:street:fr': 7325,
             'addr:street:mono': 173648,
             'addr:street:nl': 7324,
             'city_total': 85347,
             'street_total': 395412})

## Editing names

In [12]:
# Build dictionary of fixes

city_fix = build_dict(bxl, 'addr:city')
street_fix = build_dict(bxl, 'addr:street')

In [15]:
# fix streets

fix_osm(bxl, fix_address, 'addr:street', **street_fix)

In [16]:
# fix cities

fix_osm(bxl, fix_address, 'addr:city', **city_fix)

In [18]:
# Number of tags with different keys and values for cities and streets - after editing. More details in the report 
# (osm_brussels_Report.mk file).

process_names(bxl)

defaultdict(int,
            {'addr:city:bi': 61778,
             'addr:city:fr': 7285,
             'addr:city:mono': 23569,
             'addr:city:nl': 7285,
             'addr:street:bi': 255797,
             'addr:street:fr': 7325,
             'addr:street:mono': 139615,
             'addr:street:nl': 7324,
             'city_total': 85347,
             'street_total': 395412})

The editing changed names of 17,781 (61778 - 43,997) cities and 34,033 (255,797 - 221,764) streets. 

### Cities - fixing spelling

In [19]:
# addr:city
# print out cities in the dataset to inspect possible inconsistencies in spelling etc. for tag 'addr:city'

explore_names(bxl, 'addr:city')

defaultdict(int,
            {'1390': 1,
             '17': 1,
             '1850': 1,
             '3220': 1,
             '5': 1,
             'Aalst': 69,
             'Affligem': 3,
             'Afligem': 4,
             'Alsemberg': 1,
             'Anderlecht - Anderlecht': 17492,
             'Anderlecht;Forest - Vorst': 2,
             'Asse': 50,
             'Auderghem - Oudergem': 72,
             'Baal': 5,
             'Baardegem': 2,
             'Battel': 1,
             'Beersel': 3,
             'Begijnendijk': 5,
             'Berchem-Sainte-Agathe - Sint-Agatha-Berchem': 392,
             'Berlare': 7,
             'Bertem': 10,
             'Bierbeek': 5,
             'Bierges': 19,
             'Blanden': 9,
             'Bonheiden': 6,
             'Boortmeerbeek': 31,
             'Borchtlombeek': 1,
             'Bousval': 1,
             "Braine l'Alleud": 1058,
             "Braine-l'Alleud": 113,
             'Braine-le-Chateau': 4,
             'Brussegem':

In [4]:
# print out parent and child tags for values with more than one names in it (eg. Anderlecht;Forest - Vorst')

two_cities = re.compile(r'(\w;\w)')
process_problems(bxl, 'addr:city', two_cities, parents = True)

------
{'k': 'addr:city', 'v': 'Anderlecht;Forest - Vorst'}
{'k': 'operator', 'v': 'BNP Paribas Fortis'}
{'k': 'amenity', 'v': 'bank'}
{'k': 'name', 'v': 'BNP Paribas Fortis'}
{'changeset': '19284216', 'uid': '42033', 'timestamp': '2013-12-05T07:31:23Z', 'lon': '4.3264567', 'version': '7', 'user': 'Tbj', 'lat': '50.8239929', 'id': '356622931'}
------
{'k': 'addr:city', 'v': 'Uccle;Brussels'}
{'k': 'wikidata', 'v': 'Q1686386'}
{'k': 'operator', 'v': 'Foundation David and Alice van Buuren'}
{'k': 'building', 'v': 'yes'}
{'k': 'website', 'v': 'http://www.museumvanbuuren.be'}
{'k': 'tourism', 'v': 'museum'}
{'k': 'name:nl', 'v': 'Museum van Buuren'}
{'k': 'name:fr', 'v': 'Musee Alice et David van Buuren'}
{'k': 'name:en', 'v': 'Museum David and Alice van Buuren'}
{'k': 'source', 'v': 'http://www.museumvanbuuren.be/en/pratiques.php'}
{'k': 'name', 'v': 'Musee van Buuren Museum'}
{'ref': '2337598248'}
{'ref': '2337598245'}
{'ref': '2337598204'}
{'ref': '2337598174'}
{'ref': '2337598186'}
{'r

defaultdict(int,
            {'Anderlecht;Forest - Vorst': 2,
             'Kessel-Lo;Wilsele': 1,
             'Uccle;Brussels': 1})

In [25]:
# addr:city
# dictionary of fixes for cities, based on the list of all cities for key 'addr:city' (above)

spelling_fix_city = \
{
'1390': ';INVALID', # to invalidate names for cities with digits only
'17': ';INVALID',
'1850': ';INVALID',
'3220': ';INVALID',
'5': ';INVALID',    
'Afligem':  'Affligem', 
'Afflingem':  'Affligem',     
'Anderlecht;Forest - Vorst': 'Forest - Vorst', 
"Braine-l Alleud": "Braine l'Alleud",
"Braine-l'Alleud": "Braine l'Alleud",
'Brussel Evere': 'Evere',
'Brussels': 'Bruxelles - Brussel',
'Bruxelles /Brussel': 'Bruxelles - Brussel',
'Bruxelles-Brussels': 'Bruxelles - Brussel',
'Bruxelles Uccle': 'Uccle - Ukkel',
'Erps Kwerps': 'Erps-Kwerps',
'Heist o/d Berg': 'Heist op den Berg',
'Heist-Op-Den-Berg': 'Heist op den Berg',
'Heist-op-den Berg': 'Heist op den Berg',
'Heist-op-den-Berg': 'Heist op den Berg',
'Kapelle-op-den-Bosch': 'Kapelle-op-den-Bos',
'Kapelle-op-den-bos': 'Kapelle-op-den-Bos', 
'Kessel-Lo;Wilsele': 'Leuven',
'Louvain la Neuve': 'Louvain-la-neuve',
'Louvain-La-Neuve': 'Louvain-la-neuve',
'Louvain-la-Neuve': 'Louvain-la-neuve',
'Ophain-Bois-Seigneur-Isaac ': 'Ophain-Bois-Seigneur-Isaac',
'RIXENSART': 'Rixensart',
'Rhode-St-Genese': 'Rhode-Saint-Genese',
'Rhode-Saint-Genese': 'Sint-Genesius-Rode - Rhode-Saint-Genese',
'RotselaaR': 'Rotselaar',
'Sint-Pietersleeuw': 'Sint-Pieters-Leeuw',
'Sint-Stevens-Woluve': 'Sint-Stevens-Woluwe',
'Sint-Ulriks-Kapelle': 'Sint-Ulriks Kapelle',
'Sint-genesius-rode':  'Sint-Genesius-Rode - Rhode-Saint-Genese',
'Sint-Gillis-Bij-Dendermonde': 'Sint-Gillis-bij-Dendermonde',
'Uccle;Brussels': 'Uccle - Ukkel',
'Vieux-Gemappe': 'Vieux-Genappe',
'Ville de Bruxelles - Stad Brussel': 'Bruxelles - Brussel',
'WAUTHIER-BRAINE': 'Wauthier-Braine',
'WAVRE': 'Wavre',
'Woluwe-Saint-Lambert - Sint-Lambrecht-Woluwe': 'Woluwe-Saint-Lambert - Sint-Lambrechts-Woluwe' ,
'Woluwe-St-Pierre': 'Woluwe-Saint-Pierre',
'Woluwe-Saint-Pierre': 'Woluwe-Saint-Pierre - Sint-Pieters-Woluwe',
'Zaventen': 'Zaventem',
'aalst': 'Aalst',
'affligem': 'Affligem',
'bertem': 'Bertem',
"braine-l'Alleud": "Braine l'Alleud",
'bruxelles': 'Bruxelles - Brussel',
'groot-Bijgaarden': 'Groot-Bijgaarden',
'halle': 'Halle',
'huldenberg': 'Huldenberg',
'keerbergen': 'Keerbergen',
'limal': 'Limal',
'ninove': 'Ninove',
'opwijk': 'Opwijk',
'overijse': 'Overijse',
'relegem': 'Relegem',
'ternat': 'Ternat',
'tremelo': 'Tremelo',
'uccle': 'Uccle - Ukkel',
'watermael-Boitsfort - Watermaal-Bosvoorde': 'Watermael-Boitsfort - Watermaal-Bosvoorde',
'werchter': 'Werchter',
'wolvertem': 'Wolvertem'
}

# addr:city
# fix spellings of cities

fix_osm(bxl, fix_address, 'addr:city', **spelling_fix_city)

In [20]:
# addr:city:fr
# print out cities in the dataset to inspect possible inconsistencies in spelling etc. for tag 'addr:city:fr'

explore_names(bxl, 'addr:city:fr')

defaultdict(int, {'Bruxelles': 7267, 'Rhode-Saint-Genese': 17, 'bruxelles': 1})

In [17]:
# addr:city:fr
# dictionary of fixes for cities, based on the list of all cities for key 'addr:city:fr' (above)

spelling_fix_city_fr = {'bruxelles': 'Bruxelles'} 

fix_osm(bxl, fix_address, 'addr:city:fr', **spelling_fix_city_fr)

In [21]:
# addr:city:nl
# print out cities in the dataset to inspect possible inconsistencies in spelling etc. for tag 'addr:city:nl'

explore_names(bxl, 'addr:city:nl')

defaultdict(int, {'Brussel': 7268, 'Sint-Genesius-Rode': 17})

### Streets - problem characters

In [3]:
# addr:street
# print out streets with problem characters

lower_first = re.compile(r'^[a-z]'

process_problems(bxl, 'addr:street', lower_first)

defaultdict(int,
            {'abdij Van Vlierbeek': 1,
             'avenue Guillaume gilbert': 1,
             'avenue Mercator': 1,
             'avenue Roi Albert 1er': 1,
             'avenue du seigle': 1,
             'boulevard Anspach - Anspachlaan': 1,
             'chaussee de wavre': 1,
             'chemin des Sages': 1,
             'chemin du grand bois': 1,
             "d'Hoogvorstlaan": 38,
             'de Buisseretlaan': 39,
             'de Kerchovelaan': 35,
             'de Quirinilaan': 19,
             'de Salmlaan': 1,
             'de Selliersstraat': 36,
             'de Tilbourgstraat': 13,
             'della Faillestraat': 18,
             'droeshoutstraat': 1,
             'kalkestraat': 2,
             'kapeldreef': 1,
             'kasteelstraat': 1,
             'kerkpleinweg': 1,
             'kleine Daalstraat': 1,
             'milleniumstraat': 2,
             "name=Chaussee d'Anvers - Antwerpsesteenweg": 118,
             'nieuwelaan': 1,
       

In [20]:
# addr:street
# print out streets with problem characters

problem_char = re.compile(r'[=\+&<>;\"\?%#$@\,\t\r\n]')

process_problems(bxl, 'addr:street', problem_char)

defaultdict(int,
            {'57+': 2,
             'Beverdijk?': 1,
             'Chaussee de Ninove - Ninoofsesteenweg;Chaussee de Ninove - Ninoofse Steenweg': 1,
             'Dreve de "La Brise" - "La Brise"(Dreef)': 7,
             'Gr. Egm. & Hoornlaan': 1,
             'Heirweg; Haststraat': 1,
             'Place Carnoy;Place Jean-Baptiste Carnoy - Jean-Baptiste Carnoyplein': 2,
             "Route de l'Empire;Rue de l'Empire": 6,
             'Sluis; Donklaan': 7,
             "name=Chaussee d'Anvers - Antwerpsesteenweg": 118,
             'ouvain - Leuvensesteenweg;Chaussee de Louvain - Leuvensesteenweg': 9})

In [21]:
# addr:street
# print out streets with digits

digit = re.compile(r'\d')

process_problems(bxl, 'addr:street', digit)

defaultdict(int,
            {'18': 2,
             '48': 1,
             '57+': 2,
             '7de Liniestraat': 25,
             'Avenue Albert 1er': 7,
             'Avenue Roi Albert 1er': 1,
             'Baalsebaan 283': 1,
             'Edingsesteenweg 247': 1,
             'Grand Route (N4)': 1,
             'Hoogstraat 2a': 1,
             'Kasteeldreef 22': 8,
             'Lenniksestraat 2': 1,
             'Rue Albert 1er': 1,
             'Wijngaardstraat 38': 1,
             'Z.3. Doorvled': 1,
             'Z.5 Mollem': 35,
             'Zone 3 Doornveld': 2,
             'Zone 3 Doornveld 50': 3,
             'Zone 5 Mollem': 1})

In [26]:
# addr:street
# dictionary of fixes for street, based on the lists above

spelling_fix_street = \
{    
'18': ';INVALID', # to invalidate names for streets with digits only 
'48': ';INVALID',
'Baalsebaan 283': 'Baalsebaan',
'Edingsesteenweg 247': 'Edingsesteenweg',
'Hoogstraat 2a': 'Hoogstraat',
'Kasteeldreef 22': 'Kasteeldreef',
'Lenniksestraat 2': 'Lenniksestraat',
'Wijngaardstraat 38': 'Wijngaardstraat',
'Zone 3 Doornveld 50': 'Z.3. Doornveld',
'Zone 3 Doornveld': 'Z.3. Doornveld',
'Z.3. Doorvled': 'Z.3. Doornveld',
'Zone 5 Mollem': 'Z.5. Mollem',
'Beverdijk?': 'Beverdijk', 
'Chaussee de Ninove - Ninoofsesteenweg;Chaussee de Ninove - Ninoofse Steenweg': 'Chaussee de Ninove - Ninoofsesteenweg',
'Place Carnoy;Place Jean-Baptiste Carnoy - Jean-Baptiste Carnoyplein': \
    'Place Jean-Baptiste Carnoy - Jean-Baptiste Carnoyplein',
"Route de l'Empire;Rue de l'Empire": "Rue de l'Empire",
'Sluis; Donklaan': 'Donklaan',
"name=Chaussee d'Anvers - Antwerpsesteenweg": "Chaussee d'Anvers - Antwerpsesteenweg",
'ouvain - Leuvensesteenweg;Chaussee de Louvain - Leuvensesteenweg': 'Chaussee de Louvain - Leuvensesteenweg'
}


# get a dictionary of streets with lower case letter at the beginning

lower_case = process_problems(bxl, 'addr:street', lower_first)

# add to the dictionary of fixex the list of streets with lower case first letter
for key in lower_case:
    # exception for Dutch 'de'
    if (key[0:2] != "d'" and key[0:2] != 'de') and (key[0:6] != 'ouvain'and key[0:4] != 'name'): 
    
        spelling_fix_street[key] = key[0].upper() + key[1:]

In [28]:
# additional fixes in the dictionary
spelling_fix_street['chemin du grand bois'] = 'Chemin du Grand Bois'
spelling_fix_street['avenue du seigle'] = 'Avenue du Seigle'
spelling_fix_street['chaussee de wavre'] = 'Chaussee de Wavre - Steenweg op Waver'
spelling_fix_street['woluwe dal'] = 'Woluwedal'

In [29]:
# addr:street
# fix spellings of streets

fix_osm(bxl, fix_address, 'addr:street', **spelling_fix_street)

In [11]:
# addr:steet:fr
# print out streets with problem characters in the dataset for tag 'addr:city:fr'

process_problems(bxl, 'addr:street:fr', problem_char)

defaultdict(int, {})

In [12]:
# addr:steet:fr
# print out streets with lower case at the beginning for tag 'addr:street:fr'

process_problems(bxl, 'addr:street:fr', lower_first)

defaultdict(int, {'rue Gretry': 1})

In [30]:
# addr:street:fr
# print out streets with digits for tag 'addr:street:fr'

process_problems(bxl, 'addr:street:fr', digit)

defaultdict(int, {})

In [13]:
# addr:steet:fr
# fix spelling of streets 

spelling_fix_street_fr = {'rue Gentry': 'Rue Gentry'}

fix_osm(bxl, fix_address, 'addr:street:fr', **spelling_fix_street_fr)

In [14]:
# addr:steet:nl
# print out streets with problem characters in the dataset for tag 'addr:street:nl'

process_problems(bxl, 'addr:street:nl', problem_char)

defaultdict(int, {})

In [15]:
# addr:steet:nl
# print out streets with lower case at the beginning for tag 'addr:street:nl'

process_problems(bxl, 'addr:street:nl', lower_first)

defaultdict(int, {'de Witte de Haelenstraat': 29})

In [32]:
# addr:steet:nl
# print out streets with digits for tag 'addr:street:nl'

process_problems(bxl, 'addr:street:nl', digit)

defaultdict(int, {})

No need to fix any names in tag 'addr:street:nl'

In [6]:
# addr:postcode
# print out postcodes with non-digit characters

problem_postcodes = re.compile(r'[\D]') 

process_problems(bxl, 'addr:postcode', problem_postcodes)

defaultdict(int,
            {'1000;1020;1120;1130': 1,
             '1070;1190': 1,
             '1300;1301': 2,
             '1330;1331;1332': 1,
             '1340;1341;1342;1348': 1,
             '1420;1421;1428': 1,
             '1470;1471;1472;1473;1474;1476': 1,
             '1740 ternat': 1,
             '1741 Ternat': 1,
             '1755 Gooik-Oetingen': 1,
             '1780 WEMMEL': 1,
             '2220;2221;2222;2223': 1,
             '3000;3001;3010;3012;3018': 1,
             '3050 Oud-Heverlee': 1,
             '3050;3051;3052;3053;3054': 1,
             '3060 Bertem': 1,
             '3060;3061': 1,
             '3070;3071;3078': 1,
             '3110;3111;3118': 3,
             '3120 Tremelo': 1,
             '3120;3128': 1,
             '3210;3211;3212': 1,
             '3220;3221': 1,
             '61 1348': 1,
             '9300;9308;9310;9320': 1,
             '9400;9401;9402;9403;9404;9406': 1,
             '9450;9451': 1,
             '9470;9472;9473': 1,
    

In [41]:
# addr:postcode
# print out postcodes with less than four digits

short_postcodes = re.compile(r'^\d{0,3}$') 

process_problems(bxl, 'addr:postcode', short_postcodes, parents = True)

------
{'k': 'addr:postcode', 'v': '701'}
{'k': 'addr:street', 'v': 'IJsbergstraat'}
{'k': 'addr:city', 'v': 'Dilbeek'}
{'k': 'building', 'v': 'house'}
{'ref': '3610556589'}
{'ref': '3610556581'}
{'ref': '3610556582'}
{'ref': '3610556585'}
{'ref': '3610556587'}
{'ref': '3610556596'}
{'ref': '3610556589'}
{'changeset': '39289861', 'uid': '2123963', 'timestamp': '2016-05-13T11:50:00Z', 'version': '2', 'user': 'Diabolix', 'id': '355446598'}


defaultdict(int, {'701': 1})

In [42]:
# addr:postcode
# print out postcodes with more than four digits

long_postcodes = re.compile(r'^\d{5,10}$') 

process_problems(bxl, 'addr:postcode', long_postcodes, parents = True)

------
{'k': 'addr:postcode', 'v': '10303'}
{'k': 'addr:country', 'v': 'BE'}
{'k': 'addr:street', 'v': 'Boulevard Lambermont - Lambermontlaan'}
{'k': 'addr:city', 'v': 'Schaerbeek - Schaarbeek'}
{'k': 'building', 'v': 'yes'}
{'k': 'type', 'v': 'multipolygon'}
{'ref': '228736300', 'role': 'inner', 'type': 'way'}
{'ref': '228736299', 'role': 'outer', 'type': 'way'}
{'changeset': '16827307', 'uid': '681967', 'timestamp': '2013-07-04T19:48:02Z', 'version': '1', 'user': 'FantAntonio99', 'id': '3064372'}


defaultdict(int, {'10303': 1})

In [46]:
# addr:postcode
# dictionary of fixes for postcodes, based on the lists above

spelling_fix_postcode = \
{
'10303': '1030', # fixing postcode with one redundant digit 
'701': '1701',   # fixing postcode with one less digit 
'1740 ternat': '1740',
'1741 Ternat': '1741',
'1755 Gooik-Oetingen': '1755',
'1780 WEMMEL': '1780',
'3050 Oud-Heverlee': '3050',
'3060 Bertem': '3060',
'3120 Tremelo': '3120',
'61 1348': '1348',
'B-1348': '1348',
'B-1831': '1831',
'B1500': '1500',
'BE': ';INVALID', # invalidate values with invalid post codes
'Grimbergen': ';INVALID',
'Holsbeek': ';INVALID',
'World Customs Wrganization ': ';INVALID'
}  

In [47]:
# addr:postcode
# fix spelling of postcodes

fix_osm(bxl, fix_address, 'addr:postcode', **spelling_fix_postcode)

In [49]:
# postal_code
# print out postcodes with non-digit characters

problem_postcodes = re.compile(r'[\D]') 

process_problems(bxl, 'postal_code', problem_postcodes)

defaultdict(int, {})

In [50]:
# postal_code
# print out postcodes with less than four digits

process_problems(bxl, 'postal_code', short_postcodes, parents = True)

defaultdict(int, {})

In [51]:
# postal_code
# print out postcodes with more than four digits

process_problems(bxl, 'postal_code', long_postcodes, parents = True)

defaultdict(int, {})

No need to fix any code in tag 'postal_code'

## Import into SQL database

In [36]:
# export the osm_file into csv files.

from export import *

OSM_PATH = bxl

if __name__ == '__main__':
    process_map(OSM_PATH, validate = False)

In [37]:
# create sqlite3 database

import sqlite3

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()

# create tables

    cur.execute('''
        CREATE TABLE nodes (
            id INTEGER,
            lat REAL,
            lon REAL,
            user TEXT,
            uid INTEGER,
            version TEXT, 
            changeset INTEGER,
            timestamp TEXT);
            ''')
    
    cur.execute('''
        CREATE TABLE node_tags (
            id INTEGER,
            key TEXT,
            value TEXT,
            type TEXT,
            FOREIGN KEY (id) REFERENCES nodes (id));
            ''')

    cur.execute('''
        CREATE TABLE ways (
            id INTEGER,
            user TEXT,
            uid INTEGER,
            version TEXT, 
            changeset INTEGER,
            timestamp TEXT);
            ''')

    cur.execute('''
        CREATE TABLE way_nodes (
            id INTEGER,
            node_id key INTEGER,
            position INTEGER,
            FOREIGN KEY (id) REFERENCES ways (id));
            ''')

    cur.execute('''
        CREATE TABLE way_tags (
            id INTEGER,
            key TEXT,
            value TEXT,
            type TEXT,
            FOREIGN KEY (id) REFERENCES ways (id));
            ''' )
conn.commit()   

# import of the csv files into the database was made through the command line shell:

# $ sqlite3 brussels.db
#     sqlite> .mode csv
#     sqlite> .import nodes.csv nodes
#     sqlite> .import nodes_tags.csv node_tags
#     sqlite> .import ways.csv ways
#     sqlite> .import ways_nodes.csv way_nodes
#     sqlite> .import ways_tags.csv way_tags

## Database overview


In [38]:
# Numbers of nodes

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM nodes;")
    results = cur.fetchall()
    print results

[(4589143,)]


In [39]:
# total numbers of nodes, including those from ways

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT
                  (SELECT COUNT(*) FROM nodes) + 
                  (SELECT COUNT(*) FROM way_nodes);''')
    results = cur.fetchall()
    print results

[(11054570,)]


In [13]:
# Number of ways

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM ways;")
    results = cur.fetchall()
    print results

[(795223,)]


In [40]:
# number of unique users

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT COUNT(DISTINCT(users.uid))          
                   FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) users;''')
    results = cur.fetchall()
    print results

[(2537,)]


In [41]:
# top ten most active users

import pprint 

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT names.user, COUNT(*) as num
                   FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) names
                   GROUP BY names.user
                   ORDER BY num DESC
                   LIMIT 10;''')
    results = cur.fetchall()
    pprint.pprint(results) 

[(u'TAA', 918281),
 (u'lodde1949', 762899),
 (u'eMerzh', 625829),
 (u'foxandpotatoes', 350266),
 (u'Polyglot', 222496),
 (u'FantAntonio99', 213569),
 (u'Paul-Andre Duchesne', 175319),
 (u'Potato_Spirit', 167486),
 (u'escada', 157010),
 (u'Scapor', 145232)]


In [54]:
# number of public transport stops (bus stops, tram stops, underground and railway stations)

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT value, COUNT(*)
                   FROM (SELECT key, value from node_tags 
                   UNION ALL SELECT key, value from way_tags)
                   WHERE (value = 'tram_stop' OR value = 'bus_stop') OR (value = 'station' AND key = 'railway')
                   GROUP BY value;''')
    results = cur.fetchall()
    print results 

[(u'bus_stop', 9659), (u'station', 255), (u'tram_stop', 607)]


In [53]:
# number of murals in Brussels 

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT COUNT(*)
                   FROM node_tags
                   WHERE value = 'mural' 
                   GROUP BY value;''')
    results = cur.fetchall()
    print results

[(50,)]


In [52]:
import pprint

# Creators of comic books heroes who have their mural (pig wall painting) in Brussels

with sqlite3.connect(r'brussels.db') as conn:
    cur = conn.cursor()
    cur.execute('''SELECT value, COUNT(*) AS artists
                   FROM (SELECT id FROM node_tags WHERE value = 'mural') AS murals
                   JOIN node_tags 
                   ON murals.id = node_tags.id
                   WHERE key = 'title'
                   GROUP BY value
                   ORDER BY artists DESC;''')
    results = cur.fetchall()
    pprint.pprint(results) 

[(u'Ric Hochet', 2),
 (u'Tintin', 2),
 (u'Asterix et Obelix', 1),
 (u'Billy the Cat', 1),
 (u'Blake et Mortimer', 1),
 (u'Blondin et Cirage', 1),
 (u'Bob et Bobette', 1),
 (u'Boule et Bill', 1),
 (u'Broussaille', 1),
 (u'Caroline Baldwin', 1),
 (u'Cori', 1),
 (u'Corto Maltese', 1),
 (u'Cubitus', 1),
 (u'Gaston Lagaffe', 1),
 (u'Gil Jourdan', 1),
 (u'Isabelle et Calendula', 1),
 (u'Jojo', 1),
 (u'Kiekeboe', 1),
 (u"L'Ange de Sambre", 1),
 (u'La Vache', 1),
 (u'La patrouille des Castors', 1),
 (u'Le Chat', 1),
 (u'Le Jeune Albert', 1),
 (u'Le Passage', 1),
 (u'Le Petit Spirou', 1),
 (u'Le Scorpion', 1),
 (u'Le roi des mouches', 1),
 (u'Les reves de Nick', 1),
 (u'Lincoln', 1),
 (u'Lucky Luke', 1),
 (u'Martine', 1),
 (u'Monsieur Jean', 1),
 (u'Natacha', 1),
 (u'Neron', 1),
 (u'Odilon Verjus', 1),
 (u'Olivier Rameau', 1),
 (u"Passe-moi l'ciel", 1),
 (u'Quick et Flupke', 1),
 (u'Titeuf', 1),
 (u'Victor Sackville', 1),
 (u'XIII', 1),
 (u'Yoko Tsuno', 1)]
